# Step 3: Floods Cap Haitien GTFS Accessibility Analysis
Running travel time to closest facility for health facilities, schools, and markets

In [114]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [115]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np
from shapely.geometry import Point

In [116]:
import os, sys

In [117]:
# Get reference to GOSTNets
import sys
sys.path.append(r'C:\repos\GOSTnets')
import GOSTnets as gn

## Load graphs
### In order to have a better comparison with the flooded scenario notebook, we are loading the flooded graph as original_G

In [118]:
# original graph
original_G = nx.read_gpickle(r"temp\flooded_clipped_cap_haitien_walk_w_ferries_via_osmnx.pickle")

In [119]:
# read back your graphs from step 2 from your saved pickle
G_walk = nx.read_gpickle(r"temp\clipped4_cap_haitien_walk_w_ferries_via_osmnx_origins_w_time_adv_snap.pickle")
                                       
G_walk_w_gtfs = nx.read_gpickle(r"temp\4_G_walk_adv_snap_cap_haitien_gtfs_merged_impute_walk_service0001.pickle")

In [120]:
graphs = {'G_walk': G_walk, 'G_walk_w_gtfs': G_walk_w_gtfs}

## Load Data

In [121]:
# load destinations
health_destinations = gpd.read_file(r"input_folder\clipped_cap_haitien_health_pts2.shp")
school_destinations = gpd.read_file(r"input_folder\clipped_cap_haitien_schools_DPCE_20180709.shp")
market_destinations = gpd.read_file(r"input_folder\clipped_cap_haitien_markets.shp")
#shops_and_amenities = gpd.read_file(r"output_folder\osm_infrastructure\osm_shops_and_amenities.shp")

In [122]:
destinations_dict = {'health_destinations': health_destinations, 'school_destinations': school_destinations, 'market_destinations': market_destinations}

In [123]:
health_destinations

,Code2018,Facility n,Department,Vilcom,Commune,Facility t,MGA,Geolocatio,Geolocat_1,geometry
0,476,HôPital Universitaire Justinien,Nord,11,Cap-Haitien,hopital universitaire,publique,19.762251,-72.206200,POINT (-72.20620 19.76225)
1,466,Centre De Sante De Labadie (Cds),Nord,11,Cap-Haitien,centre de sante sans lit,publique,19.774759,-72.249733,POINT (-72.24973 19.77476)
2,486,Centre De Sante Quartier Morin,Nord,12,Quartier-Morin,centre de sante avec lit,publique,19.697697,-72.157616,POINT (-72.15762 19.69770)
3,516,Centre De Sante Morne Rouge (Dispensaire),Nord,22,Plaine-du-Nord,dispensaire/centre communautaire de sante,publique,19.710573,-72.265930,POINT (-72.26593 19.71057)
4,525,Centre De Santé La Providence De Lory,Nord,23,Milot,centre de sante sans lit,prive sans but lucratif,19.697908,-72.226212,POINT (-72.22621 19.69791)
5,3023,Clinique Sainte Croix Health Care,Nord,22,Plaine-du-Nord,centre de sante sans lit,prive sans but lucratif,19.729578,-72.222923,POINT (-72.22292 19.72958)
6,471,Fosref Lakay,Nord,11,Cap-Haitien,centre de sante sans lit,prive sans but lucratif,19.758741,-72.200340,POINT (-72.20034 19.75874)
7,463,Unité De Lutte Pour La Santé,Nord,11,Cap-Haitien,centre de sante sans lit,prive sans but lucratif,19.773748,-72.211250,POINT (-72.21125 19.77375)
8,2153,Centre Medical Three Strands,Nord,12,Quartier-Morin,centre de sante sans lit,prive sans but lucratif,19.717573,-72.179749,POINT (-72.17975 19.71757)
9,472,Dispensaire Caritas,Nord,11,Cap-Haitien,centre de sante sans lit,prive a but lucratif,19.754929,-72.205063,POINT (-72.20506 19.75493)


## Load Origins this way because using Advanced Snapping
### In order to have a better comparison with the flooded scenario notebook, we are loading the flooded graph origins as the originNodes. Remember in Step 1, the advanced snap function was run on the flooded walk graph with a threshold of 150 meters. Therefore the origin points within 150 meters of a road on the flooded walk graph were saved.

In [124]:
# read in origin_nodes from Step 1
originNodes = gpd.read_file(r"temp/flooded_clipped_origin_nodes_walk4.csv", GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO")

C:\Users\war-machine\anaconda3\envs\gostnets3_8\lib\site-packages\geopandas\geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [125]:
originNodes['node_ID'] = pd.to_numeric(originNodes['node_ID'])
originNodes['VALUE'] = pd.to_numeric(originNodes['VALUE'])

In [126]:
originNodes_list = list(originNodes['node_ID'])

In [127]:
originNodes_list

[1110000002,
 1110000003,
 1110000004,
 1110000005,
 1110000006,
 1110000015,
 1110000016,
 1110000017,
 1110000018,
 1110000019,
 1110000020,
 1110000021,
 1110000022,
 1110000023,
 1110000039,
 1110000040,
 1110000041,
 1110000042,
 1110000043,
 1110000044,
 1110000045,
 1110000046,
 1110000047,
 1110000048,
 1110000049,
 1110000050,
 1110000065,
 1110000066,
 1110000067,
 1110000068,
 1110000069,
 1110000070,
 1110000071,
 1110000072,
 1110000073,
 1110000074,
 1110000075,
 1110000076,
 1110000077,
 1110000078,
 1110000079,
 1110000080,
 1110000081,
 1110000103,
 1110000104,
 1110000105,
 1110000106,
 1110000107,
 1110000108,
 1110000109,
 1110000110,
 1110000111,
 1110000112,
 1110000113,
 1110000114,
 1110000115,
 1110000116,
 1110000117,
 1110000118,
 1110000119,
 1110000120,
 1110000121,
 1110000122,
 1110000123,
 1110000124,
 1110000125,
 1110000126,
 1110000127,
 1110000155,
 1110000156,
 1110000157,
 1110000158,
 1110000159,
 1110000160,
 1110000161,
 1110000162,
 1110000163,

In [128]:
len(originNodes_list)

6608

#### For each destination for each type of graph do an accessibility analysis
We are going to snap destinations to the original graph because we don't want the destinations snapping to a newly created origin node or to a GTFS line in the merged graph. We are also building statistics tables.

In [129]:
d2 = {'label_graph':[],'label_destination':[],'avg_travel_time_minutes':[]}

for G in graphs.items():
    for destination in destinations_dict.items():
        d = {'label':[],'population':[]}
        # snap the destinations to the road graph
        snapped_destinations = gn.pandana_snap(original_G, destination[1], source_crs = 'epsg:4326', target_crs = 'epsg:32619')
        destinationsNodes = list(snapped_destinations['NN'].unique())
        #print("print destinationsNodes")
        #print(destinationsNodes)
        
        #check destinationsNodes in graph
        
        # Calculate OD Matrix
        OD_matrix = gn.calculate_OD(G[1], originNodes_list, destinationsNodes, fail_value=-1, weight='length')
        
        if np.count_nonzero(OD_matrix < 0) > 1:
            print(f"warning: number of failed routes is: {np.count_nonzero(OD_matrix <0)}")
        
        # calculate accessibility
        # For each row, the closest facility is the smallest value in the row
        closest_facility_per_origin = OD_matrix.min(axis=1)
        results = pd.DataFrame([originNodes_list, closest_facility_per_origin]).transpose()
        colName = "travel_time_to_closest_facility"
        results.columns = ['node_ID', colName]
        
        # weighted average
        #https://pbpython.com/weighted-average.html
        avg_trip_time_weighted = (results['travel_time_to_closest_facility'] * originNodes['VALUE']).sum() / originNodes['VALUE'].sum()
        avg_trip_time = results['travel_time_to_closest_facility'].mean()
        
        #avg_trip_time = np.mean(closest_facility_per_origin)
        print(f"weighted average trip time for closest {destination[0]} with the {G[0]} graph is: {avg_trip_time_weighted/60} minutes")
        print(f"average trip time for closest {destination[0]} with the {G[0]} graph is: {avg_trip_time/60} minutes")
        
        output = originNodes.copy()
        output = pd.merge(output, results, on="node_ID")
        # convert travel_time_to_closest_facility to number
        output["travel_time_to_closest_facility"] = pd.to_numeric(output["travel_time_to_closest_facility"])
        output["trav_t_min"] = output["travel_time_to_closest_facility"] / 60
                
        # build statistics table
        
        d2['label_graph'].append(f'{G[0]}')
        d2['label_destination'].append(f'{destination[0]}')
        d2['avg_travel_time_minutes'].append(f'{avg_trip_time_weighted/60}')
        
        # 0-15min
        print("0-15min")
        print(output[output['travel_time_to_closest_facility'] <= 900]['VALUE'].sum())
        d['label'].append('0-15min population sum')
        d['population'].append(output[output['travel_time_to_closest_facility'] <= 900]['VALUE'].sum())
        # 15-30min
        print("15-30min")
        print(output[(output['travel_time_to_closest_facility'] > 900) & (output['travel_time_to_closest_facility'] <= 1800)]['VALUE'].sum())
        d['label'].append('15-30min population sum')
        d['population'].append(output[(output['travel_time_to_closest_facility'] > 900) & (output['travel_time_to_closest_facility'] <= 1800)]['VALUE'].sum())
        # 30-45min
        print("30-45min")
        print(output[(output['travel_time_to_closest_facility'] > 1800) & (output['travel_time_to_closest_facility'] <= 2700)]['VALUE'].sum())
        d['label'].append('30-45min')
        d['population'].append(output[(output['travel_time_to_closest_facility'] > 1800) & (output['travel_time_to_closest_facility'] <= 2700)]['VALUE'].sum())
        # 45-60min
        print("45-60min population sum:")
        print(output[(output['travel_time_to_closest_facility'] > 2700) & (output['travel_time_to_closest_facility'] <= 3600)]['VALUE'].sum())
        d['label'].append('45-60min')
        d['population'].append(output[(output['travel_time_to_closest_facility'] > 2700) & (output['travel_time_to_closest_facility'] <= 3600)]['VALUE'].sum())
        # 60min or greater
        print("60min or greater")
        print(output[(output['travel_time_to_closest_facility'] > 3600)]['VALUE'].sum())
        d['label'].append('60min or greater population sum')
        d['population'].append(output[(output['travel_time_to_closest_facility'] > 3600)]['VALUE'].sum())
        
        d_df = pd.DataFrame(d)
        d_df['population_pct'] = round(d_df.population / d_df.population.sum() * 100,2)
        
        # save a CSV of Table
        d_df.to_csv(fr"output_folder\clipped4_table_cap_haitien_accessibility_adv_snap_{destination[0]}_{G[0]}.csv")
        
        # save a shapefile...
        destinations_gpd = gpd.GeoDataFrame(output, crs = "epsg:4326", geometry = 'geometry')
        destinations_gpd.to_file(fr"output_folder\clipped4_cap_haitien_accessibility_adv_snap_{destination[0]}_{G[0]}.shp")
        
d_df2 = pd.DataFrame(d2)
# save a CSV of Table
d_df2.to_csv(fr"output_folder\table_cap_haitien_accessibility_adv_snap4.csv")


weighted average trip time for closest health_destinations with the G_walk graph is: 14.296255794350815 minutes
average trip time for closest health_destinations with the G_walk graph is: 22.717388263611603 minutes
0-15min
173313.9167041
15-30min
81093.40725387
30-45min
17790.056333020002
45-60min population sum:
4087.68072089
60min or greater
478.85772980999997


<ipython-input-129-2c7ec2f9713f>:82: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  destinations_gpd.to_file(fr"output_folder\clipped4_cap_haitien_accessibility_adv_snap_{destination[0]}_{G[0]}.shp")


weighted average trip time for closest school_destinations with the G_walk graph is: 6.067681557850035 minutes
average trip time for closest school_destinations with the G_walk graph is: 15.842371117663093 minutes
0-15min
250026.48497775997
15-30min
20611.26036286
30-45min
4861.72828496
45-60min population sum:
1165.93367833
60min or greater
98.51143778


<ipython-input-129-2c7ec2f9713f>:82: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  destinations_gpd.to_file(fr"output_folder\clipped4_cap_haitien_accessibility_adv_snap_{destination[0]}_{G[0]}.shp")


weighted average trip time for closest market_destinations with the G_walk graph is: 43.737477243564705 minutes
average trip time for closest market_destinations with the G_walk graph is: 57.04807068136621 minutes
0-15min
6675.879079820001
15-30min
48152.03312596
30-45min
106793.51750328
45-60min population sum:
75787.39690285
60min or greater
39355.092129779994


...

<ipython-input-129-2c7ec2f9713f>:82: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  destinations_gpd.to_file(fr"output_folder\clipped4_cap_haitien_accessibility_adv_snap_{destination[0]}_{G[0]}.shp")


weighted average trip time for closest school_destinations with the G_walk_w_gtfs graph is: 5.781732698705841 minutes
average trip time for closest school_destinations with the G_walk_w_gtfs graph is: 14.547506675374695 minutes
0-15min
252737.57158192
15-30min
18836.55121178
30-45min
4444.3844837
45-60min population sum:
670.38788147
60min or greater
75.02358281999999


<ipython-input-129-2c7ec2f9713f>:82: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  destinations_gpd.to_file(fr"output_folder\clipped4_cap_haitien_accessibility_adv_snap_{destination[0]}_{G[0]}.shp")


weighted average trip time for closest market_destinations with the G_walk_w_gtfs graph is: 32.22310582835167 minutes
average trip time for closest market_destinations with the G_walk_w_gtfs graph is: 40.00134403376102 minutes
0-15min
8619.08599091
15-30min
130063.1908341
30-45min
101640.76529119999
45-60min population sum:
27975.278463150004
60min or greater
8465.59816233


<ipython-input-129-2c7ec2f9713f>:82: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  destinations_gpd.to_file(fr"output_folder\clipped4_cap_haitien_accessibility_adv_snap_{destination[0]}_{G[0]}.shp")
